# File Load

### png file load(OCR)

In [2]:
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText

model_path = "nanonets/Nanonets-OCR-s"

model = AutoModelForImageTextToText.from_pretrained(
    model_path, 
    torch_dtype="auto", 
    device_map="auto", 
    # attn_implementation="flash_attention_2"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)


def ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=4096):
    prompt = """Extract the text from the above document as if you were reading it naturally. Return the tables in html format. Return the equations in LaTeX representation. If there is an image in the document and image caption is not present, add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>. Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>. Page numbers should be wrapped in brackets. Ex: <page_number>14</page_number> or <page_number>9/22</page_number>. Prefer using ☐ and ☑ for check boxes."""
    image = Image.open(image_path)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)
    
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]


Loading checkpoint shards: 100%|██████████| 2/2 [00:46<00:00, 23.01s/it]
Some parameters are on the meta device because they were offloaded to the disk.


In [ ]:
import os
import glob
import re
from PIL import Image

def extract_number(filename):
    # 파일 이름에 포함된 숫자 부분만 찾아서 반환
    numbers = re.findall(r'(\d+)', filename)
    return int(numbers[-1]) if numbers else -1 # 숫자가 여러 개 있을 경우 뒤의 숫자 사용

input_dir = "./data/중소기업은행_금융시장 동향_20231231"  # 원본 이미지 폴더
output_dir = "./data/중소기업은행_금융시장 동향_20231231/resized_images"  # 변환 이미지 저장 폴더
os.makedirs(output_dir, exist_ok=True)

# 모든 png 파일 탐색
images_files = glob.glob(os.path.join(input_dir, "*.png"))

# 자연수 기준 정렬 (1, 11, 12, 2, 3 순서 방지)
image_files.sort(key=lambda x: extract_number(os.path.basename(x)))

for img_path in image_files:
    try:
        # 이미지 열기 및 resize
        image = Image.open(img_path)
        resized_image = image.resize((image.width // 2, image.height // 2))

        # 저장 파일명 : 원본과 동일하게, 저장 디렉토리에 저장
        base = os.path.splitext(os.path.basename(img_path))[0]
        save_path = os.path.join(output_dir, f"{base}.png")
        resized_image.save(save_path)
        print(f"Saved resized image: {save_path}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {img_path} ({e})")

Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_1.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_2.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_3.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_4.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_5.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_6.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_7.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_8.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_ᄇ

In [ ]:
input_dir = "./data/예금보험공사_금융상품 동향_20240401/resized_images"    # OCR 대상 이미지 폴더
output_txt = "./data/예금보험공사_금융상품 동향_20240401/금융동향분석_전체.txt"

# 모든 png 파일 탐색
image_files = glob.glob(os.path.join(input_dir, "*.png"))

# 자연수 기준 정렬
image_files.sort(key=lambda x: extract_number(os.path.basename(x)))

with open(output_txt, "w", encoding="utf-8") as out_f:
    for idx, img_path in enumerate(image_files, 1):
        print(f"OCR 진행 중: {img_path}")
        result = ocr_page_with_nanonets_s(img_path, model, processor, max_new_tokens=2048)
        out_f.write(f"\n\n--- Page {idx} ({os.path.basename(img_path)}) ---\n\n")
        out_f.write(result)
        print(f"페이지 {idx} 결과 누적 완료")

print(f"[완료] 전체 보고서가 '{output_txt}'에 저장되었습니다.")

### hwp file load

In [6]:
from langchain_teddynote.document_loaders import HWPLoader

# HWP Loader 객체 생성
loader = HWPLoader("./data/중소기업은행_금융시장 동향_20231231/자금운용부모닝커피와마켓(20230102)_’23년 금융시장 리스크 점검.hwpx")

# 문서 로드
docs = loader.load()

In [8]:
docs[0].page_content

'   년   월   일  ’23년 금융시장 리스크 점검 지난해 전례 없는 고강도의 기준금리 인상으로 금융시장은 큰 변동성1)을 보였는데, 새해를 시작하며 ’23년에 주목해 볼 주요 리스크 요인을 짚어보고자 함.(주요 리스크)① 고금리 부작용 - 물가가 확실하게 안정되기 전까지 고금리 정책 유지 전망2)  - 높은 가계부채, 한계기업 부담가중, 자산가격(부동산 등) 하락, 유동성경색 등 우려② 지정학적 리스크 - 러-우 전쟁 장기화에 따른 안보 및 에너지위기, 러시아 돌발위험 잠복 - 중국-대만, 사우디-이란 분쟁가능성, ’24년 대선을 앞둔 미국 대응에 따른 불확실성③ 분절화(탈세계화) - - 패권경쟁 심화( : 리쇼어링, IT ․ 금융 중제재 vs  : 기술자립으로 대응) - 자국 우선주의, 블록화 등 글로벌 교역환경 변화는 세계경제에 부정적3)(시사점) 상기의 리스크 요인들이 글로벌 경제회복을 지연시킬 우려도 있으나 이를 선제적으로 대비하고 위기를 기회로 삼는다면, 우리나라가 경기 연착륙에 성공하는 희망의 한 해가 될 수 있을 것으로 기대.     1) ’22년 금융시장 변동 ( 기준금리 +225bp 인상하여 3.25%,  기준금리 +425bp 인상하여 4.50%)  (단위 : pt, 원, %)’21년연중 최저 / 최고’22년전년대비KOSPI2,977.652,134.77 / 3,010.772,236.40△741.25 (△24.9%)S&P5004,766.183,491.58 / 4,818.623,839.50△926.68 (△19.4%)달러/원 환율1,188.81,185.6 / 1,444.21,264.5+75.7  (+6.4%)국고채3년 금리1.7981.795 / 4.5553.722+192bp     2) ’23년 기준금리 상단 전망치 :  3.50~3.75%,  5.00~5.25%     3) 분절화 시뮬레이션 결과, 무역량 감소와 자원의 비효율적 배분으로 인해 전세계 실질소득 △5% 감소 예상(출처 : IMF)   숫자로 보는 뉴스      44.930일 

### txt file load

In [3]:
from langchain_community.document_loaders import TextLoader

# 텍스트 로더 생성
loader = TextLoader("data/블로그_텍스트/blog_text1.txt")

# 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

문서의 수: 1

[메타데이터]

{'source': 'data/블로그_텍스트/blog_text1.txt'}

========= [앞부분] 미리보기 =========

“월급만으로 미래 준비하고 계신가요?”
요즘 같은 고물가 시대에 단순한 저축만으로는 안정적인 미래를 기대하기 어렵습니다.
하지만 바쁜 직장 생활 속에서도 실천할 수 있는 재테크 루틴은 분명히 존재합니다.
이번 글에서는
직장인이 지금 당장 실천 가능한 현실적 재테크 전략 6가지
를 소개합니다.
[1] 통장 쪼개기: 월급 통장을 적극적으로 관리하라
많은 직장인들이 월급이 들어오는 통장을 하나로만 관리합니다.
하지만 목적에 따라 통장을 구분하면
불필요한 소비를 줄이고 자산 흐름을 파악하기 쉬워집니다.
통장 종류
목적
급여 통장
월급 수령 전용 (자동이체 최소화)
고정지출 통장
카드, 공과금, 보험 등 고정비용 이체용
저축 통장
매달 자동 이체로 저축
투자 통장
소액 투자, ETF, 펀드 등 자산 운용
[2] 월 10만 원부터 시작하는 소액 투자
많은 직장인들이 투자를 시작하기를 두려워합니다. 그러나 투자 경험이 없더라도
소액부터 시작하는 투자
는 필수입니다.
추천 투자 방법
적립식 ETF


### 디렉토리 자동 load

In [4]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"

text_loader_kwargs = {"autodetect_encoding": True}

loader = DirectoryLoader(
    path,
    glob="**/*.txt",
    loader_cls=TextLoader,
    silent_errors=True,
    loader_kwargs=text_loader_kwargs,
)
docs = loader.load()

In [5]:
print(docs[0].page_content[:500])

​
​
안녕하세요, 재테크하는제인입니다.
최근 미국의 지니어스 법안이 통과되면서 가상화폐 시장이 활기를 띠고 있는 가운데 대표적인 안전자산인 금 값이 잠시 주춤했었지요.
​
원래 금은 안전자산이기 때문에 장기 보유 하고 있는 투자자들이 많고 글로벌 위기가 닥쳤을 때는 시세가 급상승 하는 경우도 많았습니다.
​
그렇다면 지금은 완전한 글로벌 리스크 해소가 되었기 때문에 시세가 떨어진걸까요? 아닙니다.
​
당연히 지금 현재의 상황은 예전보다 전쟁이 심화되거나 하진 않지만 분명한 리스크가 존재하는데도 3개월째 계속 유지중인 이유가 있겠지요.
​
오늘은 그래서 금투자방법 중에서도 금현물에 투자하는 국내 금 ETF 4가지를 비교해보고 각각의 전략도 고려 해보겠습니다.
​
1. 금시세
​
​
현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.
​
2월에 g당 168,500원으로 고점을 찍은 이후 약 10% 정도 빠졌다고


In [6]:
print(docs[72].page_content[:500])

사회초년생 재테크
5. 입출금, 적금, 예금, CMA,
파킹통장 바로 알기
글·사진 유니콘
안녕하세요, 유니콘입니다!
성인이 되어 은행을 처음 이용하면
생각보다 많은 금융상품에
뭘 선택해야하는지 헷갈리더라구요.
​
그래서 사회초년생이라면
알고 만들어야하는
입출금통장, 적금, 예금
CMA, 파킹통장의 특징과
활용법 정리해드릴게요.
입출금통장
입출금통장은 해당 은행 이용 시
필수로 만들어야하는 상품으로
언제든지 돈을 넣고 뺄 수 있지만,
이자가 거의 없거나 매우 낮아요.
​
그래서 보통 월급을 받은 후
생활비 관리나 자동이체 용도로만
활용하는 통장이랍니다.
​
기존에 사용하던 것이 있다면
입출금만을 위해서
개설할 필요는 없어요.
이곳은 돈이 지나가는 곳이지
머무는 곳으로 쓰면 아깝습니다.
​
CMA통장
C
ash
M
anagement
A
ccount의
줄임말로 증권사에서 운영하는
종합자산관리계좌예요.
​
일반 은행 입출금통장처럼
수시로 입출금이 가능한데
예치한 돈을 증권사가
단기 금융상


# text split

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=500,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=100,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,
)

In [8]:
chunks = text_splitter.split_documents(docs)

In [9]:
chunks

[Document(metadata={'source': 'data/블로그_텍스트/blog_text60.txt'}, page_content='\u200b\n\u200b\n안녕하세요, 재테크하는제인입니다.\n최근 미국의 지니어스 법안이 통과되면서 가상화폐 시장이 활기를 띠고 있는 가운데 대표적인 안전자산인 금 값이 잠시 주춤했었지요.\n\u200b\n원래 금은 안전자산이기 때문에 장기 보유 하고 있는 투자자들이 많고 글로벌 위기가 닥쳤을 때는 시세가 급상승 하는 경우도 많았습니다.\n\u200b\n그렇다면 지금은 완전한 글로벌 리스크 해소가 되었기 때문에 시세가 떨어진걸까요? 아닙니다.\n\u200b\n당연히 지금 현재의 상황은 예전보다 전쟁이 심화되거나 하진 않지만 분명한 리스크가 존재하는데도 3개월째 계속 유지중인 이유가 있겠지요.\n\u200b\n오늘은 그래서 금투자방법 중에서도 금현물에 투자하는 국내 금 ETF 4가지를 비교해보고 각각의 전략도 고려 해보겠습니다.\n\u200b\n1. 금시세\n\u200b\n\u200b\n현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.\n\u200b'),
 Document(metadata={'source': 'data/블로그_텍스트/blog_text60.txt'}, page_content='\u200b\n1. 금시세\n\u200b\n\u200b\n현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.\n\u200b\n2월에 g당 168,500원으로 고점을 찍은 이후 약 10% 정도 빠졌다고 보면 됩니다. 그런데 이미 가격이 많이 올랐기 때문에 10%라고 하더라도 꽤 시세가 빠진 것을 확인 할 수 있습니다.\n\u200b\n그런데 그 이후로 3개월 정도 횡보하고 있는 수순이라 과연 횡보가 길어질지 아니면 상승할지는 지켜봐야 할 것 같습니다.\n\u200b\n\u200b\n국제 금

# Embedding

In [10]:
from dotenv import load_dotenv
import os


load_dotenv()

True

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    dimensions = 1024
)

In [12]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# 문자열 리스트로 생성
db = FAISS.from_documents(
    chunks,
    embeddings
)

In [13]:
db.docstore.__dict__

{'_dict': {'113b1848-b6aa-460d-8449-fd0c0ba85bae': Document(id='113b1848-b6aa-460d-8449-fd0c0ba85bae', metadata={'source': 'data/블로그_텍스트/blog_text60.txt'}, page_content='\u200b\n\u200b\n안녕하세요, 재테크하는제인입니다.\n최근 미국의 지니어스 법안이 통과되면서 가상화폐 시장이 활기를 띠고 있는 가운데 대표적인 안전자산인 금 값이 잠시 주춤했었지요.\n\u200b\n원래 금은 안전자산이기 때문에 장기 보유 하고 있는 투자자들이 많고 글로벌 위기가 닥쳤을 때는 시세가 급상승 하는 경우도 많았습니다.\n\u200b\n그렇다면 지금은 완전한 글로벌 리스크 해소가 되었기 때문에 시세가 떨어진걸까요? 아닙니다.\n\u200b\n당연히 지금 현재의 상황은 예전보다 전쟁이 심화되거나 하진 않지만 분명한 리스크가 존재하는데도 3개월째 계속 유지중인 이유가 있겠지요.\n\u200b\n오늘은 그래서 금투자방법 중에서도 금현물에 투자하는 국내 금 ETF 4가지를 비교해보고 각각의 전략도 고려 해보겠습니다.\n\u200b\n1. 금시세\n\u200b\n\u200b\n현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.\n\u200b'),
  'a7ffa698-1f25-46cb-a307-ea1fe2a3f808': Document(id='a7ffa698-1f25-46cb-a307-ea1fe2a3f808', metadata={'source': 'data/블로그_텍스트/blog_text60.txt'}, page_content='\u200b\n1. 금시세\n\u200b\n\u200b\n현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.\n\u200b\n2월에 g당 168,500원으로 고점을 찍은 이후 

In [19]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4.1-mini"),
    retriever=db.as_retriever()
)
answer = qa.run("금 한돈에 얼마야?")
print("🧠 답변:", answer)

🧠 답변: 현재 금 한돈의 정확한 가격은 제공된 정보에 없어서 알려드리기 어렵습니다. 금 가격은 시시각각 변동되므로, 최신 금 시세를 확인하려면 금융 뉴스나 금 거래 전문 사이트, 또는 은행의 금 관련 상품 페이지를 참고하시는 것이 좋습니다.
